## Exploration and cleansing of yelp_review.csv

In [1]:
import pandas as pd
import numpy as np
import re
import sweetviz as sv
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

### Importing chunk of only a thousand rows, and taking two of them as training and testing dataset

Apparently the computer here at my job is a bit of a POTATOEE  **¯\\_ _( ツ )_ _/¯**

In [2]:
reviews = pd.read_csv(r'C:\Users\lucas.corbanez\Documents\GitHub\yelp_reviews_nltk\data\yelp_review.csv', chunksize = 10 ** 3)

In [3]:
train_rev = reviews.get_chunk()
test_rev = reviews.get_chunk()

In [4]:
train_rev.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [5]:
test_rev.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
1000,B4sSnDIAzHUh8YSFu1LNeg,BytRWk8X1OelSgwwfXd8Aw,SNn_I8VMds60C4bSXCyUtw,5,2015-11-29,Every dish we tried was cooked to perfection w...,2,1,3
1001,P22ibdsG1tryITxKEK_a4Q,BytRWk8X1OelSgwwfXd8Aw,rfa_0AOmpwLTj32RL2Zvyw,5,2014-11-06,yum yum yum!\n\nWent with a party of four - we...,1,0,1
1002,SR6d_AmgGBK10ieEHsqk0Q,BytRWk8X1OelSgwwfXd8Aw,ky8BMyjQm83cqq9fx_Rn3w,3,2016-05-03,I get the whole rustic coffee shop feel that i...,2,0,0
1003,ceg_wwrykBiqITCqP411kw,BytRWk8X1OelSgwwfXd8Aw,t7R-p3dhGV78dSARAFKEOA,5,2015-07-18,Amazing place for lunch.\n\nThe salads are del...,1,0,0
1004,Ft62sweGiYYZyAjinuTbVg,BytRWk8X1OelSgwwfXd8Aw,NncsqhAElh0zykYdUULByg,4,2016-01-24,The standard for decor and atmosphere for Mont...,1,0,1


In [6]:
train_rev.text[0]

"Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly."

## Text processing

As we can see, on each review we have a bunch of punctuation and meaningful words. We are going to process every sentence for each DataFrame (test and train).

The processing will occur as follows:

In [7]:
train_rev['pos_rev'] = '' #I will add a new column with the processed text
test_rev['pos_rev'] = ''

In [8]:
train_rev['pos_rev'][0] = re.sub('[^a-zA-Z\s]', '', train_rev['text'][0]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
train_rev['pos_rev'][0] = train_rev['pos_rev'][0].lower()
train_rev['pos_rev'][0] = train_rev['pos_rev'][0].replace('\n', '')
train_rev['pos_rev'][0] = train_rev['pos_rev'][0].split()

<ipython-input-8-0b5941ccf271>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rev['pos_rev'][0] = re.sub('[^a-zA-Z\s]', '', train_rev['text'][0]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
<ipython-input-8-0b5941ccf271>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rev['pos_rev'][0] = train_rev['pos_rev'][0].lower()
<ipython-input-8-0b5941ccf271>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

### Importing stopwords

Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Allow us to remove words that would not help our model.

In [9]:
nltk.download('stopwords') 
stpw = stopwords.words('english') ## Use portuguese for real languages @=(^w^Q)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas.corbanez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Stemming words with NLTK

Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”.

Here is an example:

In [10]:
ps = PorterStemmer()
 
# choose some words to be stemmed
words = ["program", "programs", "programmer", "programing", "programmers"]
 
for w in words:
    print(w, " : ", ps.stem(w))

program  :  program
programs  :  program
programmer  :  programm
programing  :  program
programmers  :  programm


In [11]:
train_rev['pos_rev'][0] = [ps.stem(word) for word in train_rev['pos_rev'][0] if word not in stpw] #stemming only the words that are not considered stopwords

<ipython-input-11-20d7a1ed8893>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rev['pos_rev'][0] = [ps.stem(word) for word in train_rev['pos_rev'][0] if word not in stpw] #stemming only the words that are not considered stopwords


In [12]:
print('REVIEW: ', '\"', train_rev.text[0], '\"\n')

print('PROCESSED:')
train_rev['pos_rev'][0]

REVIEW:  " Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. 

Staff was very helpful and friendly. "

PROCESSED:


['super',
 'simpl',
 'place',
 'amaz',
 'nonetheless',
 'around',
 'sinc',
 'still',
 'serv',
 'thing',
 'start',
 'bologna',
 'salami',
 'sandwich',
 'mustard',
 'staff',
 'help',
 'friendli']

This processing step will be done for every review on the training and testing dataset.

In [13]:
from progressbar import ProgressBar

pbar = ProgressBar()

In [14]:
for idx in pbar(train_rev.index):
    train_rev['pos_rev'][idx] = re.sub('[^a-zA-Z\s]', '', train_rev['text'][idx]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
    train_rev['pos_rev'][idx] = train_rev['pos_rev'][idx].lower()
    train_rev['pos_rev'][idx] = train_rev['pos_rev'][idx].replace('\n', '')
    train_rev['pos_rev'][idx] = train_rev['pos_rev'][idx].split()
    train_rev['pos_rev'][idx] = [ps.stem(word) for word in train_rev['pos_rev'][idx] if word not in stpw]

<ipython-input-14-b919b6935977>:2: SettingWithCopyWarning:                    |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rev['pos_rev'][idx] = re.sub('[^a-zA-Z\s]', '', train_rev['text'][idx]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
<ipython-input-14-b919b6935977>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rev['pos_rev'][idx] = train_rev['pos_rev'][idx].lower()
<ipython-input-14-b919b6935977>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [15]:
pbar = ProgressBar()

for idx in pbar(test_rev.index):
    test_rev['pos_rev'][idx] = re.sub('[^a-zA-Z\s]', '', test_rev['text'][idx]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
    test_rev['pos_rev'][idx] = test_rev['pos_rev'][idx].lower()
    test_rev['pos_rev'][idx] = test_rev['pos_rev'][idx].replace('\n', '')
    test_rev['pos_rev'][idx] = test_rev['pos_rev'][idx].split()
    test_rev['pos_rev'][idx] = [ps.stem(word) for word in test_rev['pos_rev'][idx] if word not in stpw]

<ipython-input-15-fc5b3227d08b>:4: SettingWithCopyWarning:                    |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_rev['pos_rev'][idx] = re.sub('[^a-zA-Z\s]', '', test_rev['text'][idx]) #Everything that is not letter from a-z (A-Z) will be substituted by empty string
<ipython-input-15-fc5b3227d08b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_rev['pos_rev'][idx] = test_rev['pos_rev'][idx].lower()
<ipython-input-15-fc5b3227d08b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [16]:
train_rev

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,pos_rev
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,"[super, simpl, place, amaz, nonetheless, aroun..."
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,"[small, unassum, place, chang, menu, everi, of..."
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0,"[lester, locat, beauti, neighborhood, sinc, kn..."
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0,"[love, come, ye, place, alway, need, floor, sw..."
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0,"[chocol, almond, croissant, amaz, light, butte..."
...,...,...,...,...,...,...,...,...,...,...
995,SOhWrmqcObXs8FCnQ8Z2Lw,BytRWk8X1OelSgwwfXd8Aw,IQ3esF7MvIOCbeVetW51Ng,3,2016-06-13,Came here for a lunch because we were drawn in...,3,1,1,"[came, lunch, drawn, veggi, option, food, disa..."
996,DtSh0P83P6yDXAu0DSm7LQ,BytRWk8X1OelSgwwfXd8Aw,gfWGBaMo0XVT4q8eKX8L3A,5,2015-07-26,Charming shop in little Italy. \n\nThe service...,2,0,0,"[charm, shop, littl, itali, servic, realli, ni..."
997,uNNl27dd4mVkiyq7wGvP9w,BytRWk8X1OelSgwwfXd8Aw,XQDWifQPvL2R_c3ptJ8D2g,4,2016-02-15,This place is amazing. I'm just chiming in wit...,2,0,1,"[place, amaz, im, chime, review, say, youv, go..."
998,P0ytlvNP6Wq3Xpf_dBKXQQ,BytRWk8X1OelSgwwfXd8Aw,Qa4eXuZ1IFPwnVXJcpZWtw,3,2013-10-21,This bar is part of the Air Canada Centre and ...,1,0,1,"[bar, part, air, canada, centr, amaz, ambienc,..."


In [20]:
# test_rev = test_rev.reset_index().drop(['index'], axis = 1) , Uncomment this if you have OCD (つ◉益◉)つ
test_rev

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,pos_rev
0,B4sSnDIAzHUh8YSFu1LNeg,BytRWk8X1OelSgwwfXd8Aw,SNn_I8VMds60C4bSXCyUtw,5,2015-11-29,Every dish we tried was cooked to perfection w...,2,1,3,"[everi, dish, tri, cook, perfect, complex, sub..."
1,P22ibdsG1tryITxKEK_a4Q,BytRWk8X1OelSgwwfXd8Aw,rfa_0AOmpwLTj32RL2Zvyw,5,2014-11-06,yum yum yum!\n\nWent with a party of four - we...,1,0,1,"[yum, yum, yumwent, parti, four, seat, immedi,..."
2,SR6d_AmgGBK10ieEHsqk0Q,BytRWk8X1OelSgwwfXd8Aw,ky8BMyjQm83cqq9fx_Rn3w,3,2016-05-03,I get the whole rustic coffee shop feel that i...,2,0,0,"[get, whole, rustic, coffe, shop, feel, rage, ..."
3,ceg_wwrykBiqITCqP411kw,BytRWk8X1OelSgwwfXd8Aw,t7R-p3dhGV78dSARAFKEOA,5,2015-07-18,Amazing place for lunch.\n\nThe salads are del...,1,0,0,"[amaz, place, lunchth, salad, delici, portion,..."
4,Ft62sweGiYYZyAjinuTbVg,BytRWk8X1OelSgwwfXd8Aw,NncsqhAElh0zykYdUULByg,4,2016-01-24,The standard for decor and atmosphere for Mont...,1,0,1,"[standard, decor, atmospher, montreal, dumpl, ..."
...,...,...,...,...,...,...,...,...,...,...
995,f4VBD2sDjd_9-O33gojfSQ,PVyZXgOkVtnU6966FDFhuw,vhjRuA1f4y1zRWioJViAoA,5,2013-09-11,Yoga One rocks. Doesn't matter if you are a to...,6,1,2,"[yoga, one, rock, doesnt, matter, total, newb,..."
996,9nZu5CMtLEX2yyF5FXMg8Q,PVyZXgOkVtnU6966FDFhuw,4g_DBOk8h0A5DrdGlcv02A,3,2013-04-12,"Since moving to Charlotte, I have been on a hu...",3,1,2,"[sinc, move, charlott, hunt, find, place, sati..."
997,4XSlueYl7lkM4BNaskM5VA,PVyZXgOkVtnU6966FDFhuw,mWEltFSw33nARGPLvZiAJw,4,2016-03-04,"If you are looking for legit Mexican tacos, Ta...",2,0,0,"[look, legit, mexican, taco, taco, el, nevado,..."
998,I8Vmb63E9cptO-hsgEx8aA,PVyZXgOkVtnU6966FDFhuw,mC39IrCp36QIVFRZIw9PTQ,2,2017-11-02,6:40pm seated \n6:51pm first time seeing our w...,1,0,0,"[pm, seat, pm, first, time, see, waitress, sti..."


In [22]:
train_rev.to_csv(r'data\train_rev.csv', sep = '\t')
test_rev.to_csv(r'data\test_rev.csv', sep = '\t')